# Visualize a scattering event using my WebGL code

First, clone the GitHub repo: https://github.com/ageller/Scatter_WebGL

In [ ]:
import os
import json
import pandas as pd


from IPython.display import IFrame

import http.server
import socketserver

from threading import Thread

In [ ]:
# define the directory where the code lives (this can be anywhere on your computer; just change the directory below)
directory = os.path.join(os.getcwd())

## Create some new data to replace the default data packaged with repo

For this example, I will read in some data in a csv file that I created a while ago.  (Note that this data is downsampled, and in some parts of the time sequence the trajectories do not look smooth.)

The data will need to be reformatted into a dict and output as a json with the following structure.  Each primary key is a time, and for each time there are particles that have the word "Particle" in the key name.  Each particle has at least a key of "r" with [x,y,z] values (anything else is ignored).

Below is a visual example of how the data should be formatted:

```
{
"0.1": {
  "Particle1": {
    "r" : [-44.59797843122882, -1.602469437175624, -6.039267952136012e-14],
   },
  "Particle2": {
    "r" : [21.90496690915433, 1.108629663443225, -0.01596151404716814],
   },
  "Particle3": {
    "r" : [22.6930115220745, 0.4938397737323992, 0.01596151404722853],
   }
},
"0.2": { ...
```

In [ ]:
# read in the data
df = pd.read_csv(os.path.join(directory,'data','tripcoll.csv'))
df.loc[(df['mass'] == 0), 'x'] = 'null'
df.loc[(df['mass'] == 0), 'y'] = 'null'
df.loc[(df['mass'] == 0), 'z'] = 'null'
df

In [ ]:
# reformat and output to the json file

starIDs = df['starID'].unique()
times = df['time'].unique()

newData = dict()

for t in times:
    newData[t] = {}
    for iden in starIDs:
        row = df.loc[(df['time'] == t) & (df['starID'] == iden)]
        newData[t]['Particle' + str(iden)] = {'r': row[['x','y','z']].iloc[0].tolist()}

# serialize json
json_object = json.dumps(newData, indent = 4)
 
# write to file (ScatterParts.json is the file name, and it lives in the code's data directory)
# Note: this will replace the default data set
with open(os.path.join(directory,'data','ScatterParts.json'), "w") as outfile:
    outfile.write(json_object)

In [ ]:
# define the port that you want (8000 is standard)
port = 8000

In [ ]:
# a simple class to start the http server in a thread (so that you still have access to the jupyter notebook)
class serverThread(Thread): 
    def run(self):
        handler = http.server.SimpleHTTPRequestHandler
        os.chdir(directory)
        with socketserver.TCPServer(("", port), handler) as httpd:
            print("serving at port", port)
            httpd.serve_forever()

serverThread().start()

In [ ]:
# create an iFrame to view the visualization in this notebook
IFrame("http://localhost:8000", width = 800, height = 500)